<a href="https://colab.research.google.com/github/sayaliii21/College-Enquiry-Chat-Bot/blob/master/Data_PreProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re


In [0]:
lines=open ('movie_lines.txt' ).read().split('\n')
conversations=open ('movie_conversations.txt').read().split('\n')

In [0]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line)==5:
        id2line[_line[0]]=_line[4]


In [0]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(_conversation.split(','))

In [0]:
questions = []
answers =[]
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [0]:
def clean_text (text):
    text=text.lower()
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"he's","he is",text)
    text=re.sub(r"she's","she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"\'ll"," will",text)
    text=re.sub(r"\'ve"," have",text)
    text=re.sub(r"\'re"," are",text)
    text=re.sub(r"\'d"," would",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"can't","cannot",text)
    text=re.sub(r"[-{}\"#@/;:<>()+=~|.?,]","",text)
    return text

In [0]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))


In [0]:
word2count={}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [0]:
threshold = 20
questionswords2int = {}
answerswords2int = {}
word_number = 0
      
for word,count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1

for word,count in word2count.items():
    if count >= threshold:
          answerswords2int[word] = word_number
          word_number += 1

In [0]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
    
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

answersints2word = {w_i:w for w , w_i in answerswords2int.items()}


In [0]:
for i in range (len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [0]:
question_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int['word'])
    question_into_int.append(ints)
    
    answer_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int['word'])
    answer_into_int.append(ints)

In [0]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range (1,25+1):
    for i in enumerate (question_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(question_into_int[i[0]])
            sorted_clean_answers.append(answer_into_int[i[0]])
            
